In [4]:
import os
import random
from time import sleep
import bioblend
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy import histories
#import galaxy_ie_helpers


ipy_file_path = "scikit-learn.ipynb"

def run_code(script_path, new_history_name="ml_analysis", tool_name="run_jupyter_job"):
    
    # to be used on usegalaxy.eu
    # conn = galaxy_ie_helpers.get_galaxy_connection()
    # gi = conn.gi
    # new_history = gi.histories.create_history(new_history_name)
    
    # to be used locally
    server = 'http://127.0.0.1:9090/'
    key = '7a9d91ab284dae8207afceab171c76d4'
    gi = GalaxyInstance(server, key=key)
    history = histories.HistoryClient(gi)
    new_history = history.create_history(new_history_name)
    
    # get script
    from nbformat import read, NO_CONVERT
    with open(ipy_file_path) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    notebook_script = ""
    for cell in code_cells:
        notebook_script += cell.source + "\n\n"
    target_file_name = "target-file.py"
    with open(target_file_name, "w") as f_obj:
        f_obj.write(notebook_script)
    # upload script
    uploaded_dataset = gi.tools.upload_file(target_file_name, new_history["id"])
    sleep(20)
    hist_id = new_history["id"]
    uploaded_file_path = uploaded_dataset["outputs"][0]["id"]
    tool_run = gi.tools.run_tool(hist_id, tool_name, {"inputs": {"select_file": uploaded_file_path}})
    print(tool_run["jobs"][0]["state"])
    return tool_run

tool_ret_obj = run_code(ipy_file_path, "ml_analysis_abc")

new


In [13]:
tool_ret_obj

{'implicit_collections': [],
 'jobs': [{'create_time': '2021-09-16T08:17:00.397024',
   'exit_code': None,
   'galaxy_version': '21.09',
   'history_id': '3e4e8de146a035f5',
   'id': '244c246e3628fe4f',
   'model_class': 'Job',
   'state': 'new',
   'tool_id': 'run_jupyter_job',
   'update_time': '2021-09-16T08:17:00.435598'}],
 'output_collections': [],
 'outputs': [{'create_time': '2021-09-16T08:17:00.400152',
   'data_type': 'abc.Binary',
   'deleted': False,
   'file_ext': 'onnx',
   'file_size': 0,
   'genome_build': '?',
   'hda_ldda': 'hda',
   'hid': 2,
   'history_content_type': 'dataset',
   'history_id': '3e4e8de146a035f5',
   'id': '244c246e3628fe4f',
   'metadata_dbkey': '?',
   'misc_blurb': 'queued',
   'misc_info': None,
   'model_class': 'HistoryDatasetAssociation',
   'name': 'Output file',
   'output_name': 'outfile_output',
   'peek': None,
   'purged': False,
   'state': 'new',
   'tags': [],
   'update_time': '2021-09-16T08:17:00.438325',
   'uuid': '85b71823-d52a

In [15]:
print(tool_ret_obj["jobs"][0]["state"])

new


In [4]:
!pip install galaxy_ie_helpers

  Created wheel for galaxy-ie-helpers: filename=galaxy_ie_helpers-0.2.5-py3-none-any.whl size=6946 sha256=ec6fa1b368ed7d189d9e607ac401cb5a1db9b65a5c3aaf2738549b74989bf9cc
  Stored in directory: /home/anupkumar/.cache/pip/wheels/19/ff/93/6161d75bb369e2c4cf74b702ccc54639f568a8331fb7fcd301
Successfully built galaxy-ie-helpers


In [5]:
import bioblend
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy import histories
from time import sleep
import galaxy_ie_helpers

ipy_file_path = "scikit-learn.ipynb"

def run_code(script_path, new_history_name="ml_analysis", tool_name="run_jupyter_job"):
    
    # to be used on usegalaxy.eu
    # conn = galaxy_ie_helpers.get_galaxy_connection()
    # gi = conn.gi
    # new_history = gi.histories.create_history(new_history_name)
    
    # to be used locally
    #server = 'http://127.0.0.1:9090/'
    server = "http://localhost:8080/"
    key = '7491db238c6bedf1bfadd68507c1802d'
    gi = GalaxyInstance(server, key=key)
    history = histories.HistoryClient(gi)
    new_history = history.create_history(new_history_name)
    
    # get script
    from nbformat import read, NO_CONVERT
    with open(ipy_file_path) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    notebook_script = ""
    for cell in code_cells:
        notebook_script += cell.source + "\n\n"
    target_file_name = "target-file.py"
    with open(target_file_name, "w") as f_obj:
        f_obj.write(notebook_script)
    # upload script
    uploaded_dataset = gi.tools.upload_file(target_file_name, new_history["id"])
    sleep(20)
    hist_id = new_history["id"]
    uploaded_file_path = uploaded_dataset["outputs"][0]["id"]
    tool_run = gi.tools.run_tool(hist_id, tool_name, {"inputs": {"select_file": uploaded_file_path}})
    print(tool_run["jobs"][0]["state"])
    return tool_run

tool_ret_obj = run_code(ipy_file_path, "ml_analysis_abc")

ConnectionError: Unexpected HTTP status code: 400: {"err_msg": "parameter 'select_file': specify a dataset of the required format / build for parameter", "err_code": 400008, "err_data": {"select_file": "parameter 'select_file': specify a dataset of the required format / build for parameter"}, "param_errors": {"select_file": {"message": "parameter 'select_file': specify a dataset of the required format / build for parameter", "message_suffix": "specify a dataset of the required format / build for parameter", "parameter_name": "select_file"}}}

In [3]:
import yaml
import tensorflow as tf
from nbformat import read, NO_CONVERT

script_path = "tf_model.ipynb"

with open(script_path) as fp:
    notebook = read(fp, NO_CONVERT)
cells = notebook['cells']
code_cells = [c for c in cells if c['cell_type'] == 'code']
notebook_script = """"""
for cell in code_cells:
    notebook_script += str(cell.source)
    notebook_script += "\n\n\n"

target_file_name = "target-file.py"
with open(target_file_name, "w") as f_obj:
    f_obj.write(notebook_script)

p_loaded_file = target_file_name
global_vars = dict()
locals_vars = dict()
input_file = yaml.safe_load(p_loaded_file)

def get_e():
    with open(p_loaded_file, "r") as f:
        exec(f.read(), global_vars, locals_vars)
        return global_vars, locals_vars
    
g_v, l_v = get_e()

NameError: name 'tf' is not defined